In [1]:
import pandas as pd
import numpy as np
import collections

In [2]:
# File path for first csv
schools_data= "schools_complete.csv"

# Read our first csv into Pandas as a dataframe 
schools_data_df = pd.read_csv(schools_data)

In [3]:
# Rename "name" column to schools
schools_data_df = schools_data_df.rename(index=str, columns={"name": "school"})
#schools_data_df.head()

In [4]:
# File path for second csv
student_data = "students_complete.csv"

# Read our second csv into Pandas 
student_data_df = pd.read_csv(student_data)

In [5]:
# Get a list of all of our columns for reference, in first csv
schools_data_df.columns

Index(['School ID', 'school', 'type', 'size', 'budget'], dtype='object')

In [6]:
# Get a list of all of our columns for reference, in 2nd csv
student_data_df.columns

Index(['Student ID', 'name', 'gender', 'grade', 'school', 'reading_score',
       'math_score'],
      dtype='object')

In [7]:
# Merge Dataframes and view first 5 rows of new df 
merge_table_df = pd.merge(schools_data_df, student_data_df, how="left", on="school")
merge_table_df.head()

,School ID,school,type,size,budget,Student ID,name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


In [8]:
# Drop unnecessary columns from df
merge_table_df = merge_table_df.drop("Student ID", axis=1)
merge_table_df = merge_table_df.drop("School ID", axis=1)
#merge_table_df.head()

In [9]:
# Find total number of records
total_records = len(merge_table_df)

# Find total number of schools
number_of_schools = (merge_table_df["school"]).nunique()

# Find total count of number of students 
number_students = (merge_table_df["size"]).value_counts()
number_students = number_students.sum()

# Find the total budget 
reduced_df = merge_table_df.groupby("school").agg(np.mean)
total_budget = reduced_df["budget"].sum()

# Find the average math score
average_math_score = merge_table_df.math_score.mean()

# Find the average reading score
average_reading_score = merge_table_df.reading_score.mean()

# Find the percent passing math
percent_passing_math = merge_table_df[merge_table_df["math_score"]>= 70]
percent_passing_math = len(percent_passing_math)
percent_passing_math = (percent_passing_math/total_records) * 100

# Find the percent passing reading
percent_passing_reading = merge_table_df[merge_table_df["reading_score"]>= 70]
percent_passing_reading = len(percent_passing_reading)
percent_passing_reading = (percent_passing_reading/total_records) * 100

# Find the overall passing rate
overall_passing_rate = (percent_passing_math + percent_passing_reading)/2

In [10]:
# Create new dataframe from calculations
from collections import OrderedDict
district_summary_dict = {"Total Schools": [number_of_schools],
                        "Total Students": [number_students],
                        "Total Budget": [total_budget],
                        "Average Math Score": [average_math_score],
                        "Average Reading Score": [average_reading_score],
                        "Percent Passing Math":[percent_passing_math],
                        "Percent Passing Reading":[percent_passing_reading],
                        "Overall Passing Rate":[overall_passing_rate]}
district_summary_dict = OrderedDict(sorted(district_summary_dict.items()))
new_district_summary = OrderedDict([("Total Schools", [number_of_schools]),
                        ("Total Students", [number_students]),
                        ("Total Budget", [total_budget]),
                        ("Average Math Score", [average_math_score]),
                        ("Average Reading Score", [average_reading_score]),
                        ("Percent Passing Math", [percent_passing_math]),
                        ("Percent Passing Reading", [percent_passing_reading]),
                        ("Overall Passing Rate", [overall_passing_rate])])

district_summary_df = pd.DataFrame(new_district_summary)
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
0,15,39170,24649428.0,78.985371,81.87784,74.980853,85.805463,80.393158


In [11]:
# Improve formatting and rename columns

In [12]:
# Begin the schools summary by finding the average reading and math scores per school
new_merge_df = merge_table_df.groupby("school").mean()
new_merge_df
# Reset index
new_merge_df = new_merge_df.reset_index(level=None, drop=False, inplace=False)

In [13]:
# Add column with school type via merge
school_type_df = merge_table_df.loc[:,"school":"type"]
# I don't know why I have to have .min to work, this is a non-numeric so I can't run .mean, but I don't want to run anything 
school_type_df = pd.DataFrame(school_type_df.groupby("school").min())
# Reset index so that we can use "school" as a merge column
school_type_df = school_type_df.reset_index(level=None, drop=False, inplace=False)
school_summary_table_df = pd.merge(school_type_df, new_merge_df, how="left", on="school")

In [14]:
#percent_reading_school = student_data_df.loc[:,"school":"reading_score"]
#percent_passing_reading_school = pd.DataFrame(percent_passing_reading_school.groupby("school").count())
#percent_reading_school = pd.DataFrame(percent_passing_reading_school)
#percent_passing_reading_school = percent_passing_reading_school[percent_passing_reading_school["reading_score"]>= 70]
#percent_passing_reading_school = percent_passing_reading_school.count()
#percent_passing_reading_school = percent_passing_reading_school/school_summary_table_df["size"]
#percent_passing_reading_school = student_data_df.groupby("school")
#percent_passing_reading_school.mean()
#school_percent_passing_reading = percent_passing_reading.groupby("school").count()
#percent_passing_reading = merge_table_df[merge_table_df["reading_score"] > = 70]
#school_percent_passing_reading = percent_passing_reading.groupby("school").count()
#school_percent_passing_reading_group = school_percent_passing_reading.groupby("reading_score")
#school_percent_passing_reading_group.mean()


In [15]:
# Create a new column called "Per Student Budget" and add to existing dataframe 
school_summary_table_df["Per Student Budget"] = school_summary_table_df["budget"]/school_summary_table_df["size"]
school_summary_table_df

# Create a new column called percent passing reading
school_summary_table_df["Percent Passing Reading"] = np.nan

# Create a new column called "Percent Passing Math" and add to existing dataframe 
school_summary_table_df["Percent Passing Math"] = np.nan

# Create a new column called "Overall Passing Rate" and add to existing dataframe 
#school_passing_rate = school_summary_table_df([percent_passing_math]+[percent_passing_reading]).mean()
school_summary_table_df["Overall Passing Rate"] = np.nan
school_summary_table_df

,school,type,size,budget,reading_score,math_score,Per Student Budget,Percent Passing Reading,Percent Passing Math,Overall Passing Rate
0,Bailey High School,District,4976.0,3124928.0,81.033963,77.048432,628.0,NaN,NaN,NaN
1,Cabrera High School,Charter,1858.0,1081356.0,83.975780,83.061895,582.0,NaN,NaN,NaN
2,Figueroa High School,District,2949.0,1884411.0,81.158020,76.711767,639.0,NaN,NaN,NaN
3,Ford High School,District,2739.0,1763916.0,80.746258,77.102592,644.0,NaN,NaN,NaN
4,Griffin High School,Charter,1468.0,917500.0,83.816757,83.351499,625.0,NaN,NaN,NaN
5,Hernandez High School,District,4635.0,3022020.0,80.934412,77.289752,652.0,NaN,NaN,NaN
6,Holden High School,Charter,427.0,248087.0,83.814988,83.803279,581.0,NaN,NaN,NaN
7,Huang High School,District,2917.0,1910635.0,81.182722,76.629414,655.0,NaN,NaN,NaN
8,Johnson High School,District,4761.0,3094650.0,80.966394,77.072464,650.0,NaN,NaN,NaN
9,Pena High School,Charter,962.0,585858.0,84.044699,83.839917,609.0,NaN,NaN,NaN


In [16]:
# Create a table that highlights the top 5 performing schools based on overall passing rate
# .loc then .sort

In [17]:
# Bottom performing schools

In [18]:
# Math scores for each grade, grouped by school
student_data_df_9th = student_data_df[student_data_df["grade"] == '9th']
mathbygrade_9th = student_data_df_9th.groupby("school").math_score.mean()
mathbygrade_9th = mathbygrade_9th.reset_index(level=None, drop=False, name=None, inplace=False)

student_data_df_10th = student_data_df[student_data_df["grade"] == '10th']
mathbygrade_10th = student_data_df_10th.groupby("school").math_score.mean()
mathbygrade_10th = mathbygrade_10th.reset_index(level=None, drop=False, name=None, inplace=False)

student_data_df_11th = student_data_df[(student_data_df["grade"] == '11th')]
mathbygrade_df_11th = student_data_df_11th.groupby("school").math_score.mean()
mathbygrade_df_11th = mathbygrade_df_11th.reset_index(level=None, drop=False, name=None, inplace=False)

student_data_df_12th = student_data_df[(student_data_df["grade"] == '12th')]
mathbygrade_df_12th = student_data_df_12th.groupby("school").math_score.mean()
mathbygrade_df_12th = mathbygrade_df_12th.reset_index(level=None, drop=False, name=None, inplace=False)

# Merge all new data into one dataframe
merge_mathbygrade_df = pd.merge(mathbygrade_df_11th, mathbygrade_df_12th, how="outer", on= "school", suffixes=('_11th', '_12th'))
merge_mathbygrade_df = pd.merge(mathbygrade_10th, merge_mathbygrade_df, how= "outer", on= "school")
merge_mathbygrade_df = pd.merge(mathbygrade_9th, merge_mathbygrade_df, how= "outer", on= "school")
merge_mathbygrade_df
# Change column labels
merge_mathbygrade_df = merge_mathbygrade_df.rename(columns={"math_score_x":"math_score_9th","math_score_y":"math_score_10th"})
merge_mathbygrade_df

,school,math_score_9th,math_score_10th,math_score_11th,math_score_12th
0,Bailey High School,77.083676,76.996772,77.515588,76.492218
1,Cabrera High School,83.094697,83.154506,82.765560,83.277487
2,Figueroa High School,76.403037,76.539974,76.884344,77.151369
3,Ford High School,77.361345,77.672316,76.918058,76.179963
4,Griffin High School,82.044010,84.229064,83.842105,83.356164
5,Hernandez High School,77.438495,77.337408,77.136029,77.186567
6,Holden High School,83.787402,83.429825,85.000000,82.855422
7,Huang High School,77.027251,75.908735,76.446602,77.225641
8,Johnson High School,77.187857,76.691117,77.491653,76.863248
9,Pena High School,83.625455,83.372000,84.328125,84.121547


In [19]:
# Reading scores by grade, grouped by school

student_data_df_9th = student_data_df[student_data_df["grade"] == '9th']
readingbygrade_9th = student_data_df_9th.groupby("school").reading_score.mean()
readingbygrade_9th = readingbygrade_9th.reset_index(level=None, drop=False, name=None, inplace=False)

student_data_df_10th = student_data_df[student_data_df["grade"] == '10th']
readingbygrade_10th = student_data_df_10th.groupby("school").reading_score.mean()
readingbygrade_10th = readingbygrade_10th.reset_index(level=None, drop=False, name=None, inplace=False)

student_data_df_11th = student_data_df[(student_data_df["grade"] == '11th')]
readingbygrade_11th = student_data_df_11th.groupby("school").reading_score.mean()
readingbygrade_11th = readingbygrade_11th.reset_index(level=None, drop=False, name=None, inplace=False)

student_data_df_12th = student_data_df[(student_data_df["grade"] == '12th')]
readingbygrade_12th = student_data_df_12th.groupby("school").reading_score.mean()
readingbygrade_12th = readingbygrade_12th.reset_index(level=None, drop=False, name=None, inplace=False)

# Merge all new data into one dataframe
merge_readingbygrade_df = pd.merge(readingbygrade_11th, readingbygrade_12th, how="outer", on= "school", suffixes=('_11th', '_12th'))
merge_readingbygrade_df = pd.merge(readingbygrade_10th, merge_readingbygrade_df, how= "outer", on= "school")
merge_readingbygrade_df = pd.merge(readingbygrade_9th, merge_readingbygrade_df, how= "outer", on= "school")
merge_readingbygrade_df
# Change column labels
merge_readingbygrade_df = merge_readingbygrade_df.rename(columns={"reading_score_x":"reading_score_9th","reading_score_y":"reading_score_10th"})
merge_readingbygrade_df

,school,reading_score_9th,reading_score_10th,reading_score_11th,reading_score_12th
0,Bailey High School,81.303155,80.907183,80.945643,80.912451
1,Cabrera High School,83.676136,84.253219,83.788382,84.287958
2,Figueroa High School,81.198598,81.408912,80.640339,81.384863
3,Ford High School,80.632653,81.262712,80.403642,80.662338
4,Griffin High School,83.369193,83.706897,84.288089,84.013699
5,Hernandez High School,80.866860,80.660147,81.396140,80.857143
6,Holden High School,83.677165,83.324561,83.815534,84.698795
7,Huang High School,81.290284,81.512386,81.417476,80.305983
8,Johnson High School,81.260714,80.773431,80.616027,81.227564
9,Pena High School,83.807273,83.612000,84.335938,84.591160


In [20]:
#Scores by school spending per student budget
bins = [575, 600, 620, 640, 660]
spending_names = ["575-599", "600-619", "620-639", "640-659"]
school_summary_table_df["Spending per student"] = pd.cut(school_summary_table_df["Per Student Budget"], bins, labels=spending_names)
#school_summary_table_df.head()
school_summary_table_df = school_summary_table_df.drop("size", axis = 1)
school_summary_table_reduced_df = school_summary_table_df.drop("budget", axis = 1)
#school_summary_table_reduced_df
# Create a GroupBy object based upon "Spending per student"
school_spending_group = school_summary_table_reduced_df.groupby("Spending per student")
school_spending_group.mean()

,reading_score,math_score,Per Student Budget,Percent Passing Reading,Percent Passing Math,Overall Passing Rate
Spending per student,,,,,,
575-599,83.892196,83.436210,584.80,NaN,NaN,NaN
600-619,84.044699,83.839917,609.00,NaN,NaN,NaN
620-639,82.120471,79.474551,633.40,NaN,NaN,NaN
640-659,80.957446,77.023555,650.25,NaN,NaN,NaN


In [21]:
# drop columns to get the right df , need to do it before 

# Scores by school size
bins = [0, 1750, 3250, 5000]
group_names = ["Small", "Medium", "Large"]
# Change to use newer df
new_merge_df["School Size Category"] = pd.cut(new_merge_df["size"], bins, labels=group_names)
school_size_categories = new_merge_df.groupby("School Size Category")
school_size_categories.mean()
#school_size_categories = school_size_categories.drop("size", axis=1)

,size,budget,reading_score,math_score
School Size Category,,,,
Small,1123.000000,6.986438e+05,83.881343,83.603261
Medium,2329.571429,1.437985e+06,82.676142,80.545935
Large,4592.750000,2.947240e+06,80.919864,77.063340


In [22]:
#Scores by School type
school_type_categories = school_summary_table_df.groupby("type")
school_type_categories.mean()

,budget,reading_score,math_score,Per Student Budget,Percent Passing Reading,Percent Passing Math,Overall Passing Rate
type,,,,,,,
Charter,9.126881e+05,83.896421,83.473852,599.500000,NaN,NaN,NaN
District,2.478275e+06,80.966636,76.956733,643.571429,NaN,NaN,NaN
